Antonis code with langchain

### Download documentation and add them to a list
#### https://python.langchain.com/docs/integrations/document_transformers/html2text


In [ ]:
!pip install requests beautifulsoup4
!pip install html2text



In [14]:
from langchain.document_loaders import AsyncHtmlLoader
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from langchain.document_transformers import Html2TextTransformer
urls = "https://ucy-linc-lab.github.io/fogify/"

def crawlulr(start_url):
  url_contents=[]
  # Send a GET request to the URL
  response = requests.get(start_url)

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the HTML content
      soup = BeautifulSoup(response.text, 'html.parser')

      # Find all 'a' tags (links) in the page
      links = soup.find_all('a')

      # Extract and print all URLs
      for link in links:
          # Get the href attribute of each 'a' tag
          href = link.get('href')

          # Join the URL if it's relative
          full_url = urljoin(start_url, href)

          print(f"Crawling url: {full_url}")
          loader = AsyncHtmlLoader(full_url)
          docs = loader.load()
          html2text = Html2TextTransformer()
          docs_transformed = html2text.transform_documents(docs)
          url_contents.append(docs_transformed)
  else:
      print("Failed to retrieve the page")
  return url_contents

document_content=crawlulr(urls)


Crawling url: https://ucy-linc-lab.github.io/fogify


Fetching pages: 100%|##########| 1/1 [00:00<00:00, 13.86it/s]


Crawling url: http://linc.ucy.ac.cy/


Fetching pages: 100%|##########| 1/1 [00:02<00:00,  2.04s/it]


Crawling url: https://github.com/UCY-LINC-LAB/fogify


...

Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.65s/it]


Crawling url: https://ucy-linc-lab.github.io/fogify/5g-slicer.html


Fetching pages: 100%|##########| 1/1 [00:00<00:00, 13.96it/s]


Crawling url: https://rainbow-h2020.eu/


Fetching pages: 100%|##########| 1/1 [00:03<00:00,  3.56s/it]


In [16]:
len(document_content)


44

In [17]:
document_content[3]

[Document(page_content='Skip to content Toggle navigation\n\nSign up\n\n  * Product \n\n    * Actions\n\nAutomate any workflow\n\n    * Packages\n\nHost and manage packages\n\n    * Security\n\nFind and fix vulnerabilities\n\n    * Codespaces\n\nInstant dev environments\n\n    * Copilot\n\nWrite better code with AI\n\n    * Code review\n\nManage code changes\n\n    * Issues\n\nPlan and track work\n\n    * Discussions\n\nCollaborate outside of code\n\nExplore\n\n    * All features \n    * Documentation \n    * GitHub Skills \n    * Blog \n\n  * Solutions \n\nFor\n\n    * Enterprise \n    * Teams \n    * Startups \n    * Education \n\nBy Solution\n\n    * CI/CD & Automation \n    * DevOps \n    * DevSecOps \n\nResources\n\n    * Learning Pathways \n    * White papers, Ebooks, Webinars \n    * Customer Stories \n    * Partners \n\n  * Open Source \n\n    * GitHub Sponsors\n\nFund open source developers\n\n    * The ReadME Project\n\nGitHub community articles\n\nRepositories\n\n    * Topic

In [2]:
!pip install sentence_transformers > /dev/null
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [4]:
text = "This is a test document."
query_result = embeddings.embed_query(text) #create query embeddings
#result compared to the two documents in the second document we can see the ones
#in the DB although this should be automated
doc_result = embeddings.embed_documents([text, "This is not a test document."])
doc_result

[[-0.03833852708339691,
  0.1234646737575531,
  -0.028643004596233368,
  0.053652696311473846,
  0.008845353499054909,
  -0.03983934596180916,
  -0.07300588488578796,
  0.04777134209871292,
  -0.030462512746453285,
  0.05497974529862404,
  0.08505292236804962,
  0.03665668144822121,
  -0.005319966934621334,
  -0.0022331562358886003,
  -0.060711007565259933,
  -0.027237895876169205,
  -0.01135161705315113,
  -0.04243773594498634,
  0.00912995170801878,
  0.1008155345916748,
  0.07578732818365097,
  0.06911717355251312,
  0.009857435710728168,
  -0.0018377684755250812,
  0.026249045506119728,
  0.03290243074297905,
  -0.07177434116601944,
  0.028384286910295486,
  0.06170952320098877,
  -0.052529554814100266,
  0.033661648631095886,
  0.07446809858083725,
  0.07536033540964127,
  0.03538401052355766,
  0.06713403761386871,
  0.010798031464219093,
  0.08167022466659546,
  0.01656288467347622,
  0.0328306183218956,
  0.03632565960288048,
  0.002172840293496847,
  -0.09895739704370499,
  0.